In [17]:
reccomendation().sort_values(by='output').to_csv('overall.csv')

Input you age --> 94


In [13]:
import urllib.request
import json
import math
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm
url = 'https://data.cityofnewyork.us/resource/xyye-rtrs.json'
# Fetch the data from the URL
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
coordinates_to_neighborhood = {}
for neighborhood in data:
    if neighborhood['borough'] == "Manhattan":
        lat = neighborhood['the_geom']['coordinates'][1]
        lon = neighborhood['the_geom']['coordinates'][0]
        coordinates_to_neighborhood[tuple([lat, lon])] = neighborhood['name']   

data = coordinates_to_neighborhood
# Create a list of dictionaries with longitude, latitude, and neighborhood
formatted_data = [{'longitude': k[0], 'latitude': k[1], 'neighborhood': v} for k, v in data.items()]
# Create a DataFrame
df = pd.DataFrame(formatted_data)

In [14]:
df_data = pd.read_csv('data_score.csv')
final_df = pd.merge(df, df_data, how='left', on='neighborhood')
final_df = final_df[final_df[['streeteasy_score', 'mta_score']].notnull().all(axis=1)]
weight_df = pd.read_csv('data_weight.csv')

In [15]:
def get_weight(age):
    global weight_df, q,c,o
    row = weight_df[weight_df['Age'] == age]
    q = row['q'].values[0]
    c = row['c'].values[0]
    o = row['o'].values[0]
def calculate_score(tw,dd,gr,se,mt,q,c,o):
    quality_array = np.array([tw, dd, gr])
    quality_non_nan = quality_array[~np.isnan(quality_array)]
    average_quality = np.mean(quality_non_nan)
    
    result = average_quality*q + mt*c + se*o
    result = round(result, 1)
    return result

In [11]:
def reccomendation():
    age = int(input("Input you age --> "))
    get_weight(age)
    final_df['output'] = final_df.apply(lambda row: calculate_score(row['twitter_score'], row['door_dash_score'],
                                                                   row['grocery_score'],row['streeteasy_score'],
                                                                   row['mta_score'],q,c,o), axis=1)
    sorted_df = final_df.sort_values('output', ascending=False)
    #top_3_rows = sorted_df.head(10)
    top_3_rows = sorted_df.copy()
    # Define the list of neighborhoods and their corresponding numbers
    neighborhoods = list(top_3_rows['neighborhood'])
    neighborhood_numbers = list(top_3_rows['output']) 
    corodinate_list = list(top_3_rows[['longitude', 'latitude']].to_records(index=False))
    # Define the latitude and longitude of Manhattan's center
    manhattan_lat, manhattan_lon = 40.7831, -73.9712
    # Create a map of Manhattan with CartoDB's Positron tiles
    #Without color
    m = folium.Map(location=[manhattan_lat, manhattan_lon], zoom_start=11, tiles='CartoDB Positron')
    #with color
    #m = folium.Map(location=[manhattan_lat, manhattan_lon], zoom_start=13)
    # Create a colormap based on the range of neighborhood_numbers
    color_scale = cm.LinearColormap(
    colors=["red", "yellow", "green", "blue"],
    vmin=min(neighborhood_numbers),
    vmax=max(neighborhood_numbers),
    caption="Neighborhood Numbers"
    )
    for neighborhood, number, (lat, lon) in zip(neighborhoods, neighborhood_numbers, corodinate_list):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color_scale(number),
            fill=True,
            fill_opacity=0.7
        ).add_to(m)
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(icon_size=(150,36), icon_anchor=(0,0),
            html='<div style="font-size: 8.5pt; color: black;">{0}: {1}</div>'.format(neighborhood, number))
        ).add_to(m)
        color_scale.add_to(m)
    name = "manhattan_age"+str(age)+".html"
    m.save(name)
    return final_df